Making request

In [1]:
import requests
import tensorflow as tf
import numpy as np
import json
import base64
import requests
from pprint import PrettyPrinter
import pandas as pd

In [2]:
pp = PrettyPrinter()
pp.pprint(requests.get("https://diplomatic-luck-production.up.railway.app/v1/models/cardiovaskular-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1722168646'}]}


In [3]:
df = pd.read_csv('data/cardio_data_processed.csv')
df.head(1)

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,bmi,bp_category,bp_category_encoded
0,2,168,62.0,110,80,1,1,0,0,1,0,50,21.96712,Hypertension Stage 1,Hypertension Stage 1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68205 entries, 0 to 68204
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               68205 non-null  int64  
 1   height               68205 non-null  int64  
 2   weight               68205 non-null  float64
 3   ap_hi                68205 non-null  int64  
 4   ap_lo                68205 non-null  int64  
 5   cholesterol          68205 non-null  int64  
 6   gluc                 68205 non-null  int64  
 7   smoke                68205 non-null  int64  
 8   alco                 68205 non-null  int64  
 9   active               68205 non-null  int64  
 10  cardio               68205 non-null  int64  
 11  age_years            68205 non-null  int64  
 12  bmi                  68205 non-null  float64
 13  bp_category          68205 non-null  object 
 14  bp_category_encoded  68205 non-null  object 
dtypes: float64(2), int64(11), object(2)


In [5]:
row = df.iloc[np.random.randint(0, len(df))]
test = row.to_dict()
print(test)
print(type(test))

{'gender': 1, 'height': 164, 'weight': 68.0, 'ap_hi': 130, 'ap_lo': 90, 'cholesterol': 2, 'gluc': 2, 'smoke': 0, 'alco': 0, 'active': 1, 'cardio': 1, 'age_years': 64, 'bmi': 25.282569898869724, 'bp_category': 'Hypertension Stage 1', 'bp_category_encoded': 'Hypertension Stage 1'}
<class 'dict'>


In [6]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def float64_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

In [7]:
def prepare_json(inputs:dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, int):
            feature_spec[keys] = int64_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float64_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
    
    example = tf.train.Example(
      features=tf.train.Features(feature=feature_spec)).SerializeToString()
    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]
    
    return json.dumps(
        {
            "signature_name":"serving_default",
            "instances": result
        }
    )

In [8]:
def get_predict(text):
    json_data = prepare_json(text)
    headers = {'Content-Type': 'application/json'}
    response = requests.post("https://diplomatic-luck-production.up.railway.app/v1/models/cardiovaskular-model:predict", data=json_data, headers=headers)
    predict = response.json()["predictions"][0][0]
    return predict

In [9]:
predict = get_predict(test)
print(test)
print(f'PRESENCE' if predict > 0.6 else f'ABSENCE')
print(f"Result: '{predict}' ")
# Label (0 = ABSENCE and 1 = PRESENCE)

{'gender': 1, 'height': 164, 'weight': 68.0, 'ap_hi': 130, 'ap_lo': 90, 'cholesterol': 2, 'gluc': 2, 'smoke': 0, 'alco': 0, 'active': 1, 'cardio': 1, 'age_years': 64, 'bmi': 25.282569898869724, 'bp_category': 'Hypertension Stage 1', 'bp_category_encoded': 'Hypertension Stage 1'}
PRESENCE
Result: '0.648574352' 
